In [1]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
from copy import deepcopy as dcopy,copy

from z3 import *
import grilops
import grilops.regions

In [2]:
# The grilops demo
def main():
    
      givens = [[0, 0, 0, 3, 0, 0, 0, 0, 5],
                [0, 0, 8, 3, 10, 0, 0, 5, 0],
                [0, 3, 0, 0, 0, 4, 4, 0, 0],
                [1, 3, 0, 3, 0, 0, 2, 0, 0],
                [0, 2, 0, 0, 3, 0, 0, 2, 0],
                [0, 0, 2, 0, 0, 3, 0, 1, 3],
                [0, 0, 4, 4, 0, 0, 0, 3, 0],
                [0, 4, 0, 0, 4, 3, 3, 0, 0],
                [6, 0, 0, 0, 0, 1, 0, 0, 0]]   
      sym = grilops.make_number_range_symbol_set(1, 10)
      lattice = grilops.get_rectangle_lattice(len(givens), len(givens[0]))
      sg = grilops.SymbolGrid(lattice, sym)
      rc = grilops.regions.RegionConstrainer(lattice,solver=sg.solver)

      for p in lattice.points:
        # The filled number must match the size of the region.
        sg.solver.add(sg.grid[p] == rc.region_size_grid[p])

        # The size of the region must match the clue.
        given = givens[p.y][p.x]
        if given != 0:
              sg.solver.add(rc.region_size_grid[p] == given)

    # Different regions of the same size may not be orthogonally adjacent.
        region_sizes = [n.symbol for n in lattice.edge_sharing_neighbors(rc.region_size_grid, p)]
        region_ids = [n.symbol for n in lattice.edge_sharing_neighbors(rc.region_id_grid, p)    ]
        for region_size, region_id in zip(region_sizes, region_ids):
              sg.solver.add(Implies(rc.region_size_grid[p] == region_size,rc.region_id_grid[p] == region_id))

      if sg.solve():
        sg.print()

        if sg.is_unique():
              print("Unique solution")
        else:
               print("Alternate solution")
               sg.print()
      else:
        print("No solution")
    
      return sg

sg = main()

8 8 3 3 101010105 
8 8 8 3 1010105 5 
3 3 8 10104 4 4 5 
1 3 8 3 102 2 4 5 
2 2 8 3 3 1 3 2 2 
6 6 2 2 1 3 3 1 3 
6 4 4 4 2 2 1 3 3 
6 4 2 2 4 3 3 4 4 
6 6 4 4 4 1 3 4 4 
Unique solution


In [4]:
# longhand in z3py

def neigbours(grid, i, j):
        l = []
        (rows,cols) = grid.shape
        if i-1 >= 0:
            l.append((i-1,j))
        if i+1 < rows:
            l.append((i+1,j))
        if j-1 >= 0:
            l.append((i,j-1))
        if j+1 < cols:
            l.append((i,j+1))
        return l

givens = np.array([[0, 0, 0, 3, 0, 0, 0, 0, 5],
                [0, 0, 8, 3, 10, 0, 0, 5, 0],
                [0, 3, 0, 0, 0, 4, 4, 0, 0],
                [1, 3, 0, 3, 0, 0, 2, 0, 0],
                [0, 2, 0, 0, 3, 0, 0, 2, 0],
                [0, 0, 2, 0, 0, 3, 0, 1, 3],
                [0, 0, 4, 4, 0, 0, 0, 3, 0],
                [0, 4, 0, 0, 4, 3, 3, 0, 0],
                [6, 0, 0, 0, 0, 1, 0, 0, 0]],dtype=int)     
    
N = givens.shape[0]

In [ ]:
start = time.time()    
s = Solver()

# Directed edges
edge = {}
for i in range(N):
    for j in range(N):  
        for (k,l) in neigbours(givens,i,j) :
            edge[(i,j,k,l)] = Int("e%d%d%d%d" % (i,j,k,l))
            s += Or(edge[(i,j,k,l)] ==0,edge[(i,j,k,l)]==1) 
            
# Tree 1: edges only in one direction
for i in range(N):
    for j in range(N):  
        for (k,l) in neigbours(givens,i,j):
            s += (edge[(i,j,k,l)] + edge[(k,l,i,j)]) <= 1
    
# Tree 2: each cell has at most one parent
for i in range(N):
    for j in range(N):  
        s += Sum([edge[(k,l,i,j)] for (k,l) in neigbours(givens,i,j) ]) <=1    

# Number in cell    
X = [[Int("x%d%d" % (i,j)) for j in range (N) ] for i in range (N) ]    

# Initial numbers in cell
for i in range(N):
    for j in range(N):  
        if givens[i,j] > 0:
            s += X[i][j] == int(givens[i,j])  
            
# Size of cell         
size = [[Int("s%d%d" % (i,j)) for j in range (N) ] for i in range (N) ]

# Calculate size of cell as 1 + size of children
for i in range(N):
    for j in range(N):   
        s +=  size[i][j] == Sum([If(edge[(i,j,k,l)]==1,size[k][l],0) for (k,l) in neigbours(givens,i,j)])+1
        
# For roots size of cell must be equal to number in cell
for i in range(N):
    for j in range(N):  
        s+= Implies(Sum([edge[(k,l,i,j)] for (k,l) in neigbours(givens,i,j)]) == 0 ,size[i][j]==X[i][j])      
    
# Number of children is equal to number of parent
for i in range (N) :
    for j in range (N) : 
        for (k,l) in neigbours(givens,i,j) :
            s +=Implies(edge[(i,j,k,l)] == 1,X[i][j]== X[k][l])
    
# Root of corresponding tree
root = [[Int("r%d%d" % (i,j)) for j in range (N) ] for i in range (N) ]

# Roots    
for i in range(N):
    for j in range(N):  
        s+= Implies(Sum([edge[(k,l,i,j)] for (k,l) in neigbours(givens,i,j)]) == 0 ,root[i][j]==i*N+j)      
        
# Neighboring cells with same number must be in same area        
for i in range (N) :
    for j in range (N) : 
        for (k,l) in neigbours(givens,i,j) :
            s +=Implies(X[i][j]==X[k][l],root[i][j]==root[k][l])    
    
    
        
if s.check() == sat:
    print("Solved in {:.2f} seconds".format(time.time()-start))
    m = s.model()
    r = [ [ m.evaluate(X[i][j]).as_long() for j in range(N) ] for i in range(N) ]
    print(np.array( [ [ m.evaluate(root[i][j]).as_long() for j in range(N) ] for i in range(N) ]))
    fig,ax = plt.subplots(1,1,figsize=(4,4))
    ax =sns.heatmap(np.array(r),annot=np.array(r),cbar=False,center=3,cmap="tab20_r",fmt="",linewidths=1,linecolor="grey",annot_kws={"size":16})
    ax.axis("off")
    plt.show()
    
else:
    print("failed to solve")